In [1]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.parent) +'/')
from utils.visualize import img_grid
from utils.SFCN import SFCNModel

from torchvision import transforms 

import numpy as np
import pandas as pd
import torchio as tio
import nibabel as nib

import ipywidgets as ipyw
import matplotlib.pyplot as plt
import SimpleITK as sitk
import torch
from  torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

In [2]:
Path(os.getcwd()).parent.parent

PosixPath('/home/miplab/Projects/macaw')

In [3]:
macaw_path = str(Path(os.getcwd()).parent.parent)
ukbb_path = '/home/miplab/ukbb'
ukbb_T1_warped = ukbb_path + '/T1_warped'
generated_path = ukbb_path + '/generated_images'
filtered_path = generated_path + '/median_filtered'
z_initial = 50
z_fim = 149
nsamples = 5

In [4]:
data_path = ukbb_path + '/ukbb_img.csv'
df = pd.read_csv(data_path,low_memory=False)

# Applying Median Filter

In [5]:
'''

for file in os.listdir(generated_path):
    if '_50-149.nii' in file:
        image_original = sitk.ReadImage(os.path.join(generated_path, file))
        median = sitk.MedianImageFilter()
        median.SetRadius(1)
        output = median.Execute(image_original)
        sitk.WriteImage(output, os.path.join(filtered_path, file))
'''

"\n\nfor file in os.listdir(generated_path):\n    if '_50-149.nii' in file:\n        image_original = sitk.ReadImage(os.path.join(generated_path, file))\n        median = sitk.MedianImageFilter()\n        median.SetRadius(1)\n        output = median.Execute(image_original)\n        sitk.WriteImage(output, os.path.join(filtered_path, file))\n"

In [6]:
# Getting the generated images
subjects_eid = np.array([])
cf_age = np.array([])
cf_sex = np.array([])
images_generated = []
for file in tqdm(os.listdir(generated_path)):
    if '_50-149.nii' in file:
        splits = file.split("_")
        subject_eid = int(splits[0])
        age = splits[1]
        sex = splits[2].split(".")[0]
        subjects_eid = np.append(subjects_eid,subject_eid)
        cf_age = np.append(cf_age, age)
        cf_sex = np.append(cf_sex, sex)
        image_load = nib.load(os.path.join(generated_path, file)).get_fdata()
        images_generated.append(image_load)
print(len(images_generated))

  0%|          | 0/211 [00:00<?, ?it/s]

205


In [7]:
rands = np.random.randint(0,len(images_generated),nsamples) 
images_generated = np.array(images_generated)
images_generated = images_generated[rands]
subjects_eid = subjects_eid[rands].astype(int)
cf_age = cf_age[rands]
cf_sex = cf_sex[rands]

In [8]:
# Getting the original images
images_original = []
real_age = []
real_sex = []
for individual in subjects_eid:
    real_age.append(df[df["eid"] == individual]["Age"].item())
    real_sex.append(df[df["eid"] == individual]["Sex"].item())
    image_load = nib.load(os.path.join(ukbb_T1_warped, str(individual) + ".nii.gz")).get_fdata()
    images_original.append(image_load)

In [9]:
real_age

[61.0, 68.0, 48.0, 69.0, 71.0]

In [10]:
titles_cf = [f'Age:{a}, Sex:{s}' for a,s in zip(cf_age,cf_sex)]
real_sex = ['M' if round(s) else 'F' for s in real_sex]
titles_real = [f'Age:{a}, Sex:{s}' for a,s in zip(real_age,real_sex)]

In [11]:
#cropping
generated_shape = images_generated[0].shape
original_shape = images_original[0].shape
x_initial = int((original_shape[0] - generated_shape[0])/2)
x_fim = x_initial +  generated_shape[0]
y_initial = int((original_shape[1] - generated_shape[1])/2)
y_fim = y_initial +  generated_shape[1]
images_original = [image[x_initial:x_fim,y_initial:y_fim,z_initial:z_fim+1] for image in images_original]

In [12]:
# Normalizing
for one_image in images_original:
    slices = one_image.shape[2]
    for z in range(slices):
        maxv = np.max(one_image[:,:,z])
        minv = np.min(one_image[:,:,z])
        one_image[:,:,z] = ((one_image[:,:,z] - minv) / maxv)

In [13]:
# Swaping X and Y
images_generated = [np.swapaxes(one_image,0,1) for one_image in images_generated]

In [14]:
# Mapping the difference

diff = [np.subtract(a,s) for a,s in zip(images_original,images_generated)]

In [15]:
def plot_slice(z_slice):
    order_slice = z_slice - z_initial
    plt.rcParams["figure.figsize"] = (20,5)
    img_grid([one_image[:,:,order_slice] for one_image in images_original],cols=nsamples,titles=titles_real)
    img_grid([one_image[:,:,order_slice] for one_image in images_generated],cols=nsamples,titles=titles_cf)
    img_grid([one_image[:,:,order_slice] for one_image in diff],cols=nsamples, cmap='seismic', clim=(-1,1))

In [16]:
ipyw.interact(plot_slice, 
              z_slice=ipyw.IntSlider(min=z_initial, max=z_fim, step=1, continuous_update=False, description='Image Slice:'))

interactive(children=(IntSlider(value=50, continuous_update=False, description='Image Slice:', max=149, min=50…

<function __main__.plot_slice(z_slice)>